<a href="https://colab.research.google.com/github/Levan-Danelia/FRTB/blob/main/FRTB_FDXL.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
# Cell 1: Setup and Initial Data
# Import necessary libraries
import pandas as pd
import numpy as np

# --- Initial Portfolio Data ---
# This data represents the starting positions and their sensitivities.
data = [
    {'Position_ID': 3, 'Bucket': 'GBP_USD', 'Sensitivity': -104597},
    {'Position_ID': 4, 'Bucket': 'GBP_USD', 'Sensitivity': -943522},
    {'Position_ID': 5, 'Bucket': 'JPY_USD', 'Sensitivity': 176122},
    {'Position_ID': 6, 'Bucket': 'JPY_USD', 'Sensitivity': 1461581}
]

# --- Regulatory Parameters ---
# Risk Weight for liquid FX pairs as per Article 325av
risk_weight = 0.15 / np.sqrt(2)

# Cross-Bucket Correlation for FX as per Article 325aw
gamma_medium = 0.60

# Create an initial DataFrame
portfolio_df = pd.DataFrame(data)
portfolio_df.set_index('Position_ID', inplace=True)

print("--- Initial Setup Complete ---")
print(f"Loaded {len(portfolio_df)} positions.")
print("\nInitial Portfolio:")
display(portfolio_df)

--- Initial Setup Complete ---
Loaded 4 positions.

Initial Portfolio:


,Bucket,Sensitivity
Position_ID,,
3,GBP_USD,-104597
4,GBP_USD,-943522
5,JPY_USD,176122
6,JPY_USD,1461581


In [2]:

# Cell 2: Step 2 - Calculate Net Sensitivities (sₖ)
# As per Article 325f, we net sensitivities for identical risk factors (buckets).

print("\n### Step 2: Calculate Net Sensitivities (sₖ) ###")
print("Sensitivities are grouped by their bucket and summed.")

# Group by bucket and sum sensitivities to get the net sensitivity
net_sensitivities_df = portfolio_df.groupby('Bucket').agg(
    Net_Sensitivity_sk=('Sensitivity', 'sum')
).reset_index()

display(net_sensitivities_df)


### Step 2: Calculate Net Sensitivities (sₖ) ###
Sensitivities are grouped by their bucket and summed.


,Bucket,Net_Sensitivity_sk
0,GBP_USD,-1048119
1,JPY_USD,1637703


In [3]:

# Cell 3: Step 3 - Calculate Weighted Sensitivities (WSₖ)
# Each net sensitivity is multiplied by its regulatory risk weight (RWₖ) from Article 325av.

print("\n### Step 3: Calculate Weighted Sensitivities (WSₖ) ###")
print(f"Applying regulatory risk weight of {risk_weight:.2%} to the net sensitivities.")

# Add the risk weight column
net_sensitivities_df['Risk_Weight_RWk'] = risk_weight

# Calculate the weighted sensitivity
net_sensitivities_df['Weighted_Sensitivity_WSk'] = net_sensitivities_df['Net_Sensitivity_sk'] * net_sensitivities_df['Risk_Weight_RWk']

display(net_sensitivities_df[['Bucket', 'Net_Sensitivity_sk', 'Risk_Weight_RWk', 'Weighted_Sensitivity_WSk']])


### Step 3: Calculate Weighted Sensitivities (WSₖ) ###
Applying regulatory risk weight of 10.61% to the net sensitivities.


,Bucket,Net_Sensitivity_sk,Risk_Weight_RWk,Weighted_Sensitivity_WSk
0,GBP_USD,-1048119,0.106066,-111169.807859
1,JPY_USD,1637703,0.106066,173704.634530


In [4]:

# Cell 4: Step 4 & 5 - Intra-Bucket Aggregation (Kₑ)
# For FX delta, each bucket has one risk factor, so Kb is the absolute value of WSk.

print("\n### Step 4 & 5: Intra-Bucket Aggregation (Kₑ) ###")
print("Calculating bucket-specific capital (Kₑ).")

net_sensitivities_df['Bucket_Capital_Ke'] = net_sensitivities_df['Weighted_Sensitivity_WSk'].abs()

display(net_sensitivities_df[['Bucket', 'Weighted_Sensitivity_WSk', 'Bucket_Capital_Ke']])


### Step 4 & 5: Intra-Bucket Aggregation (Kₑ) ###
Calculating bucket-specific capital (Kₑ).


,Bucket,Weighted_Sensitivity_WSk,Bucket_Capital_Ke
0,GBP_USD,-111169.807859,111169.807859
1,JPY_USD,173704.634530,173704.634530


In [5]:
# Cell 5: Step 6, 7, 8 & 9 - Across-Bucket Aggregation and Final Charge
# This function aggregates the bucket-level capital (Kₑ) to get the final
# risk class capital for a given scenario.

def calculate_final_capital(kb_df, scenario='medium'):
    """
    Calculates the final risk class capital based on Article 325f and 325h.
    """
    gamma_medium = 0.60 # From Article 325aw

    # Adjust correlation based on the scenario as per Article 325h
    if scenario == 'high':
        gamma = min(gamma_medium * 1.25, 1.0)
    elif scenario == 'low':
        gamma = max(2 * gamma_medium - 1, 0.75 * gamma_medium)
    else: # medium
        gamma = gamma_medium

    k_values = kb_df['Bucket_Capital_Ke'].values
    s_values = kb_df['Weighted_Sensitivity_WSk'].values

    # Sum of squares of Kₑ values
    sum_k_sq = np.sum(k_values**2)

    # Sum of cross-bucket terms
    cross_bucket_sum = 0
    if len(k_values) > 1:
        # This calculates 2 * gamma * S1 * S2 for the two buckets
        cross_bucket_sum = 2 * gamma * s_values[0] * s_values[1]

    # The term inside the square root is floored at zero
    final_capital_sq = max(sum_k_sq + cross_bucket_sum, 0)

    return np.sqrt(final_capital_sq)

print("\n### Step 6-9: Across-Bucket Aggregation & Final Charge ###")
print("Aggregating bucket capital using cross-bucket correlations for all three scenarios.")

# Calculate final capital for all scenarios
medium_scenario_capital = calculate_final_capital(net_sensitivities_df, scenario='medium')
high_scenario_capital = calculate_final_capital(net_sensitivities_df, scenario='high')
low_scenario_capital = calculate_final_capital(net_sensitivities_df, scenario='low')

# --- Display Summary ---
scenario_summary = pd.DataFrame([
    {'Scenario': 'Medium', 'Correlation': f"{gamma_medium:.0%}", 'Capital': medium_scenario_capital},
    {'Scenario': 'High', 'Correlation': f"{min(gamma_medium * 1.25, 1.0):.0%}", 'Capital': high_scenario_capital},
    {'Scenario': 'Low', 'Correlation': f"{max(2 * gamma_medium - 1, 0.75 * gamma_medium):.0%}", 'Capital': low_scenario_capital}
])
display(scenario_summary.set_index('Scenario'))

# --- Final Charge Calculation ---
print("\n### Final Charge Calculation ###")
final_capital_charge = scenario_summary['Capital'].max()
print(f"The final capital requirement is the highest of the three scenarios.")

summary_card = pd.DataFrame([{'Final FX Delta Capital Requirement': f"{final_capital_charge:,.0f}"}])
display(summary_card)


### Step 6-9: Across-Bucket Aggregation & Final Charge ###
Aggregating bucket capital using cross-bucket correlations for all three scenarios.


,Correlation,Capital
Scenario,,
Medium,60%,139137.245994
High,75%,116473.001032
Low,45%,158595.039255



### Final Charge Calculation ###
The final capital requirement is the highest of the three scenarios.


,Final FX Delta Capital Requirement
0,"158,595"
